## 📦 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import time

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV, mutual_info_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# LightGBM
import lightgbm as lgb

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Libraries loaded successfully!")

## 📂 Load Data

In [ ]:
# Load all CSV files
files = [
    'dataset-type-1/2021.csv',
    'dataset-type-1/2022.csv',
    'dataset-type-1/2023.csv',
    'dataset-type-1/A2021.csv',
    'dataset-type-1/A2022.csv',
    'dataset-type-1/A2023.csv'
]

dfs = []
for file in files:
    try:
        df_temp = pd.read_csv(file)
        dfs.append(df_temp)
        print(f"✅ Loaded {file}: {len(df_temp)} rows")
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Combine all data
df_raw = pd.concat(dfs, ignore_index=True)
print(f"\n📊 Total data: {len(df_raw):,} rows")
print(f"📅 Columns: {list(df_raw.columns)}")
df_raw.head()

## 🔧 Data Preprocessing

In [ ]:
# Rename columns untuk konsistensi
df = df_raw.copy()
df.columns = df.columns.str.upper()

# Standardize column names
column_mapping = {
    'KODE': 'KODE_BARANG',
    'TANGGAL': 'TANGGAL',
    'QTY_MSK': 'QTY_MSK',
    'NILAI_MSK': 'NILAI_MSK',
    'QTY_KLR': 'QTY_KLR',
    'NILAI_KLR': 'NILAI_KLR'
}

# Parse dates
df['TANGGAL'] = pd.to_datetime(df['TANGGAL'], format='%d-%m-%y', errors='coerce')

# Remove invalid dates
df = df.dropna(subset=['TANGGAL'])

# Fill missing values
df['QTY_MSK'] = df['QTY_MSK'].fillna(0)
df['QTY_KLR'] = df['QTY_KLR'].fillna(0)
df['NILAI_MSK'] = df['NILAI_MSK'].fillna(0)
df['NILAI_KLR'] = df['NILAI_KLR'].fillna(0)

print(f"✅ After preprocessing: {len(df):,} rows")
print(f"📅 Date range: {df['TANGGAL'].min()} to {df['TANGGAL'].max()}")
df.info()

## 🎯 Select Top Products

In [ ]:
# Analyze top products by output volume
top_n = 30  # Ambil top 30 products untuk efisiensi

product_volume = df.groupby('KODE_BARANG')['QTY_KLR'].sum().sort_values(ascending=False)
top_products = product_volume.head(top_n).index.tolist()

print(f"🔝 Top {top_n} products by volume:")
print(product_volume.head(top_n))

# Filter untuk top products only
df_filtered = df[df['KODE_BARANG'].isin(top_products)].copy()
print(f"\n✅ Filtered data: {len(df_filtered):,} rows")

## 🏗️ Feature Engineering

In [ ]:
# Sort by product and date
df_filtered = df_filtered.sort_values(['KODE_BARANG', 'TANGGAL']).reset_index(drop=True)

# Create daily aggregations
daily_agg = df_filtered.groupby(['KODE_BARANG', 'TANGGAL']).agg({
    'QTY_MSK': ['sum', 'mean'],
    'QTY_KLR': ['sum', 'mean'],
    'NILAI_MSK': ['sum', 'mean'],
    'NILAI_KLR': ['sum', 'mean']
}).reset_index()

# Flatten column names
daily_agg.columns = ['_'.join(col).strip('_') for col in daily_agg.columns.values]
daily_agg.columns = ['product', 'date', 'qty_in_sum', 'qty_in_mean', 
                     'qty_out_sum', 'qty_out_mean', 'value_in_sum', 
                     'value_in_mean', 'value_out_sum', 'value_out_mean']

print(f"✅ Daily aggregations: {len(daily_agg):,} rows")
daily_agg.head()

In [ ]:
# Add temporal features
daily_agg['day'] = daily_agg['date'].dt.day
daily_agg['month'] = daily_agg['date'].dt.month
daily_agg['day_of_week'] = daily_agg['date'].dt.dayofweek  # 0=Monday, 6=Sunday
daily_agg['week_of_year'] = daily_agg['date'].dt.isocalendar().week

# Add lag features (historical data)
for lag in [1, 2, 3, 7]:
    daily_agg[f'qty_in_lag{lag}'] = daily_agg.groupby('product')['qty_in_sum'].shift(lag)
    daily_agg[f'qty_out_lag{lag}'] = daily_agg.groupby('product')['qty_out_sum'].shift(lag)

# Add rolling statistics (7-day window)
daily_agg['qty_in_roll7'] = daily_agg.groupby('product')['qty_in_sum'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)
daily_agg['qty_out_roll7'] = daily_agg.groupby('product')['qty_out_sum'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

print("✅ Feature engineering completed!")
print(f"📊 Total features: {len(daily_agg.columns)}")
print(f"\nFeatures: {list(daily_agg.columns)}")

## 🎯 Prepare Train/Test Data

In [ ]:
# Define target and features
target_col = 'qty_out_sum'  # Predict current day output

# Features to use (exclude target, identifiers, and date)
exclude_cols = ['product', 'date', target_col, 'qty_out_mean', 'value_out_sum', 'value_out_mean']
feature_cols = [col for col in daily_agg.columns if col not in exclude_cols]

print(f"🎯 Target: {target_col}")
print(f"📊 Features ({len(feature_cols)}): {feature_cols}")

# Remove rows with NaN (due to lag features)
df_clean = daily_agg.dropna().copy()
print(f"\n✅ Clean data: {len(df_clean):,} rows (after removing NaN)")

# Split features and target
X = df_clean[feature_cols]
y = df_clean[target_col]

# Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False  # shuffle=False for time series
)

print(f"\n📦 Train set: {len(X_train):,} rows")
print(f"📦 Test set: {len(X_test):,} rows")
print(f"\n📊 Target statistics:")
print(f"  - Mean: {y_train.mean():.2f}")
print(f"  - Std: {y_train.std():.2f}")
print(f"  - Min: {y_train.min():.2f}")
print(f"  - Max: {y_train.max():.2f}")

## 🔬 Experiment 1: RFECV Feature Selection

In [ ]:
print("=" * 60)
print("🔬 EXPERIMENT 1: RFECV + LightGBM")
print("=" * 60)

# Base estimator for RFECV
base_estimator = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    verbose=-1
)

# RFECV - automatically finds optimal number of features
print("\n⏳ Running RFECV... (this may take a few minutes)")
start_time = time.time()

rfecv = RFECV(
    estimator=base_estimator,
    step=1,
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)

rfecv.fit(X_train, y_train)
rfecv_time = time.time() - start_time

# Get selected features
rfecv_features = X_train.columns[rfecv.support_].tolist()
print(f"\n✅ RFECV completed in {rfecv_time:.2f} seconds")
print(f"🎯 Optimal number of features: {rfecv.n_features_}")
print(f"📊 Selected features: {rfecv_features}")

# Transform data
X_train_rfecv = rfecv.transform(X_train)
X_test_rfecv = rfecv.transform(X_test)

In [ ]:
# Train final model with selected features
print("\n🏋️ Training LightGBM with RFECV features...")
start_time = time.time()

model_rfecv = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

model_rfecv.fit(X_train_rfecv, y_train)
train_time_rfecv = time.time() - start_time

# Predictions
y_pred_rfecv = model_rfecv.predict(X_test_rfecv)

# Metrics
rmse_rfecv = np.sqrt(mean_squared_error(y_test, y_pred_rfecv))
mae_rfecv = mean_absolute_error(y_test, y_pred_rfecv)
r2_rfecv = r2_score(y_test, y_pred_rfecv)

print(f"\n📊 RFECV Results:")
print(f"  ⏱️  Total time: {rfecv_time + train_time_rfecv:.2f}s")
print(f"  🎯 Features used: {rfecv.n_features_}")
print(f"  📉 RMSE: {rmse_rfecv:.4f}")
print(f"  📉 MAE: {mae_rfecv:.4f}")
print(f"  📈 R²: {r2_rfecv:.4f}")

## 🔬 Experiment 2: Mutual Information Feature Selection

In [ ]:
print("=" * 60)
print("🔬 EXPERIMENT 2: Mutual Information + LightGBM")
print("=" * 60)

# Calculate mutual information scores
print("\n⏳ Calculating Mutual Information scores...")
start_time = time.time()

mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_jobs=-1)
mi_time = time.time() - start_time

# Create feature importance dataframe
mi_df = pd.DataFrame({
    'feature': X_train.columns,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

print(f"\n✅ MI calculation completed in {mi_time:.2f} seconds")
print(f"\n📊 Top 10 features by MI score:")
print(mi_df.head(10))

# Select top k features (same number as RFECV for fair comparison)
top_k = rfecv.n_features_
mi_features = mi_df.head(top_k)['feature'].tolist()

print(f"\n🎯 Selected top {top_k} features: {mi_features}")

# Transform data
X_train_mi = X_train[mi_features]
X_test_mi = X_test[mi_features]

In [ ]:
# Train final model with MI features
print("\n🏋️ Training LightGBM with MI features...")
start_time = time.time()

model_mi = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

model_mi.fit(X_train_mi, y_train)
train_time_mi = time.time() - start_time

# Predictions
y_pred_mi = model_mi.predict(X_test_mi)

# Metrics
rmse_mi = np.sqrt(mean_squared_error(y_test, y_pred_mi))
mae_mi = mean_absolute_error(y_test, y_pred_mi)
r2_mi = r2_score(y_test, y_pred_mi)

print(f"\n📊 Mutual Information Results:")
print(f"  ⏱️  Total time: {mi_time + train_time_mi:.2f}s")
print(f"  🎯 Features used: {top_k}")
print(f"  📉 RMSE: {rmse_mi:.4f}")
print(f"  📉 MAE: {mae_mi:.4f}")
print(f"  📈 R²: {r2_mi:.4f}")

## 📊 Comparison & Results

In [ ]:
# Create comparison dataframe
results = pd.DataFrame({
    'Method': ['RFECV', 'Mutual Information'],
    'RMSE': [rmse_rfecv, rmse_mi],
    'MAE': [mae_rfecv, mae_mi],
    'R2': [r2_rfecv, r2_mi],
    'Time (s)': [rfecv_time + train_time_rfecv, mi_time + train_time_mi],
    'Num Features': [rfecv.n_features_, top_k]
})

print("\n" + "="*80)
print("📊 FINAL COMPARISON: RFECV vs Mutual Information")
print("="*80)
print(results.to_string(index=False))

# Determine winner
print("\n🏆 WINNER ANALYSIS:")
if rmse_rfecv < rmse_mi:
    print(f"  🥇 RFECV wins on RMSE ({rmse_rfecv:.4f} < {rmse_mi:.4f})")
else:
    print(f"  🥇 Mutual Information wins on RMSE ({rmse_mi:.4f} < {rmse_rfecv:.4f})")

if r2_rfecv > r2_mi:
    print(f"  🥇 RFECV wins on R² ({r2_rfecv:.4f} > {r2_mi:.4f})")
else:
    print(f"  🥇 Mutual Information wins on R² ({r2_mi:.4f} > {r2_rfecv:.4f})")

if (rfecv_time + train_time_rfecv) < (mi_time + train_time_mi):
    print(f"  ⚡ RFECV is faster ({rfecv_time + train_time_rfecv:.2f}s < {mi_time + train_time_mi:.2f}s)")
else:
    print(f"  ⚡ Mutual Information is faster ({mi_time + train_time_mi:.2f}s < {rfecv_time + train_time_rfecv:.2f}s)")

# Save results
results.to_csv('results_dataset1_pharmacy.csv', index=False)
print("\n💾 Results saved to: results_dataset1_pharmacy.csv")

## 📈 Visualization

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Dataset 1: Feature Selection Comparison (RFECV vs MI)', fontsize=16, fontweight='bold')

methods = ['RFECV', 'MI']
colors = ['#3498db', '#e74c3c']

# Plot 1: RMSE
ax1 = axes[0, 0]
ax1.bar(methods, [rmse_rfecv, rmse_mi], color=colors, alpha=0.7, edgecolor='black')
ax1.set_ylabel('RMSE', fontsize=12, fontweight='bold')
ax1.set_title('Root Mean Squared Error (Lower is Better)', fontsize=12)
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate([rmse_rfecv, rmse_mi]):
    ax1.text(i, v, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

# Plot 2: R²
ax2 = axes[0, 1]
ax2.bar(methods, [r2_rfecv, r2_mi], color=colors, alpha=0.7, edgecolor='black')
ax2.set_ylabel('R² Score', fontsize=12, fontweight='bold')
ax2.set_title('R² Score (Higher is Better)', fontsize=12)
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate([r2_rfecv, r2_mi]):
    ax2.text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

# Plot 3: Training Time
ax3 = axes[1, 0]
times = [rfecv_time + train_time_rfecv, mi_time + train_time_mi]
ax3.bar(methods, times, color=colors, alpha=0.7, edgecolor='black')
ax3.set_ylabel('Time (seconds)', fontsize=12, fontweight='bold')
ax3.set_title('Total Processing Time (Lower is Better)', fontsize=12)
ax3.grid(axis='y', alpha=0.3)
for i, v in enumerate(times):
    ax3.text(i, v, f'{v:.2f}s', ha='center', va='bottom', fontweight='bold')

# Plot 4: MAE
ax4 = axes[1, 1]
ax4.bar(methods, [mae_rfecv, mae_mi], color=colors, alpha=0.7, edgecolor='black')
ax4.set_ylabel('MAE', fontsize=12, fontweight='bold')
ax4.set_title('Mean Absolute Error (Lower is Better)', fontsize=12)
ax4.grid(axis='y', alpha=0.3)
for i, v in enumerate([mae_rfecv, mae_mi]):
    ax4.text(i, v, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('comparison_dataset1_pharmacy.png', dpi=300, bbox_inches='tight')
print("\n📊 Visualization saved to: comparison_dataset1_pharmacy.png")
plt.show()

## 🔍 Feature Importance Analysis

In [ ]:
# Compare selected features
print("\n📊 FEATURE SELECTION COMPARISON:")
print("\n🔹 RFECV Selected Features:")
for i, feat in enumerate(rfecv_features, 1):
    print(f"  {i}. {feat}")

print("\n🔹 Mutual Information Top Features:")
for i, row in mi_df.head(top_k).iterrows():
    print(f"  {i+1}. {row['feature']} (score: {row['mi_score']:.4f})")

# Find common features
common_features = set(rfecv_features) & set(mi_features)
print(f"\n🤝 Common Features ({len(common_features)}): {common_features}")

# Unique features
rfecv_only = set(rfecv_features) - set(mi_features)
mi_only = set(mi_features) - set(rfecv_features)
print(f"\n🔹 RFECV-only features: {rfecv_only}")
print(f"🔹 MI-only features: {mi_only}")

## 📝 Conclusion & Recommendations

In [ ]:
print("\n" + "="*80)
print("📝 KESIMPULAN & REKOMENDASI")
print("="*80)

# Calculate percentage differences
rmse_diff = abs(rmse_rfecv - rmse_mi) / min(rmse_rfecv, rmse_mi) * 100
r2_diff = abs(r2_rfecv - r2_mi) / max(r2_rfecv, r2_mi) * 100
time_diff = abs((rfecv_time + train_time_rfecv) - (mi_time + train_time_mi)) / min(rfecv_time + train_time_rfecv, mi_time + train_time_mi) * 100

print(f"\n📊 Performance Differences:")
print(f"  • RMSE difference: {rmse_diff:.2f}%")
print(f"  • R² difference: {r2_diff:.2f}%")
print(f"  • Time difference: {time_diff:.2f}%")

print(f"\n🎯 Recommendations:")

if rmse_rfecv < rmse_mi and r2_rfecv > r2_mi:
    print("  ✅ RFECV is BETTER for prediction accuracy")
    print("  📌 Use RFECV when: Accuracy is critical, computational time is acceptable")
elif rmse_mi < rmse_rfecv and r2_mi > r2_rfecv:
    print("  ✅ Mutual Information is BETTER for prediction accuracy")
    print("  📌 Use MI when: Need fast results with good accuracy")
else:
    print("  ⚖️ Both methods perform SIMILARLY on accuracy")
    
if (mi_time + train_time_mi) < (rfecv_time + train_time_rfecv):
    speed_ratio = (rfecv_time + train_time_rfecv) / (mi_time + train_time_mi)
    print(f"  ⚡ MI is {speed_ratio:.1f}x FASTER than RFECV")
    print("  📌 Advantage: Quick iterations, real-time applications")

print(f"\n💡 Best Practice:")
print("  1️⃣ Use MI for fast initial feature selection")
print("  2️⃣ Use RFECV for final model optimization")
print("  3️⃣ Compare both to understand feature importance from different perspectives")

print("\n✅ Analysis Complete!")